# Masks to CAD output
This notebook will produce CAD-compatible vectors in .DXF format from predicted masks and saves them to an output folder.

### Imports

In [3]:
import os
import cv2
import numpy as np
import ezdxf

### Paths
Choose whether to use predictions from U-Net or YOLO below

In [9]:
input_folder = "../data/predicted/U_Net/masks"
#input_folder = "../data/predicted/YOLO/masks"
output_folder = "../data/cad_output"

### Convert Function

In [5]:
def png_to_dxf_smooth(input_png, output_dxf, smoothing_iterations=3, epsilon_factor=0.01):
    """
    Converts a binary PNG image to a DXF file with smoothed edges.
    
    Parameters:
    - input_png (str): Path to the input binary PNG file containing the mask.
    - output_dxf (str): Path to save the output DXF file.
    - smoothing_iterations (int): Number of smoothing iterations for edge refinement.
    - epsilon_factor (float): Factor to control the contour approximation (lower = more detail).
    """
    # Load the binary PNG image
    image = cv2.imread(input_png, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Cannot open file: {input_png}")

    # Apply Gaussian Blur for smoothing
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Threshold the image to binary (black and white)
    _, binary_image = cv2.threshold(blurred_image, 128, 255, cv2.THRESH_BINARY)

    # Perform morphological smoothing (optional)
    kernel = np.ones((3, 3), np.uint8)
    smoothed_binary = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel, iterations=smoothing_iterations)

    # Find contours of the shapes in the binary image
    contours, _ = cv2.findContours(smoothed_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a new DXF document
    doc = ezdxf.new()
    msp = doc.modelspace()

    # Iterate through contours, approximate, and add smoothed polylines to the DXF
    for contour in contours:
        # Simplify the contour using contour approximation
        epsilon = epsilon_factor * cv2.arcLength(contour, True)
        approx_curve = cv2.approxPolyDP(contour, epsilon, True)

        # Convert to a list of points and scale
        points = [(point[0][0], -point[0][1]) for point in approx_curve]
        if len(points) > 1:
            msp.add_lwpolyline(points, close=True)

    # Save the DXF file
    doc.saveas(output_dxf)
    print(f"DXF file saved as: {output_dxf}")



### Execute

In [10]:
os.makedirs(output_folder, exist_ok=True)

# Iterate through all files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is a PNG
    if file_name.lower().endswith(".png"):
        input_png_path = os.path.join(input_folder, file_name)
        output_dxf_name = os.path.splitext(file_name)[0] + ".dxf"
        output_dxf_path = os.path.join(output_folder, output_dxf_name)

        # Call the function
        print(f"Processing: {input_png_path} -> {output_dxf_path}")
        png_to_dxf_smooth(input_png_path, output_dxf_path)

print("Batch conversion complete.")

Processing: ../data/predicted/YOLO/masks\A_Fia_05_mask.png -> ../data/cad_output\A_Fia_05_mask.dxf
DXF file saved as: ../data/cad_output\A_Fia_05_mask.dxf
Batch conversion complete.
